
<h1 id="Projet-2:-Social-Network-Analysis">Social Network Analysis<a class="anchor-link" href="#Projet-2:-Social-Network-Analysis">¶</a></h1>



<h1 id="Deuxième-Partie">Partie Twitter<a class="anchor-link" href="#Deuxième-Partie">¶</a></h1>



<h4 id="On-va-tout-d'abord-télécharger-la-librairie-Tweepy-qui-sert-à-travailler-avec-Twitter.">On va tout d'abord télécharger la librairie Tweepy qui sert à travailler avec Twitter.<a class="anchor-link" href="#On-va-tout-d'abord-télécharger-la-librairie-Tweepy-qui-sert-à-travailler-avec-Twitter.">¶</a></h4><h5 id="On-utilise-la-commande-suivante-dans-la-console:">On utilise la commande suivante dans la console:<a class="anchor-link" href="#On-utilise-la-commande-suivante-dans-la-console:">¶</a></h5><p>pip install tweepy</p>


In [ ]:

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline 

import pandas as pd



In [ ]:

import tweepy
 
auth = tweepy.OAuthHandler("XXXXXXXXXXXX", "XXXXXXXXXXXXXXXXXXXX")
auth.set_access_token("XXXXXXXXXXXXXX", "XXXXXXXXXXXXXXXXXXXXXXX ")
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)



In [ ]:

results = tweepy.Cursor(api.search,q = "Trump", l="en").items(400)
idTweet=[]
screenName=[]
text=[]
createdAt=[]
for r in results:
    idTweet.append(r.id)
    screenName.append(r.user.screen_name)
    text.append(r.text)
    createdAt.append(r.created_at)



In [ ]:

data=pd.DataFrame()
data['idTweet']=idTweet
data['screenName']=screenName
data['text']=text
data['createdAt']=createdAt
data['createdAt']=pd.to_datetime(data['createdAt'])
data['EST'] = data['createdAt'] - pd.Timedelta(hours=5)
data['EST'] = pd.to_datetime(data['EST'])



In [ ]:

data.head(10)



In [ ]:

allUsers=list(data['screenName'].unique())



In [ ]:

data.to_csv('results.csv',index = False,encoding='utf-8')



In [ ]:

import time
import sys



In [11]:

dataUsers=pd.DataFrame()
dataUsers['userFromName']=[]
dataUsers['userFromId']=[]
dataUsers['userToId']=[]
count=0
for name in allUsers:
    followers = []
    for page in tweepy.Cursor(api.followers_ids, screen_name=name).pages():
        followers.extend(page)
    
    currentId = api.get_user(screen_name=name).id
    currentId = [currentId] * len(followers)
    currentName = [name] * len(followers)   
    
    dataT = pd.DataFrame()
    dataT['userFromName'] = currentName
    dataT['userFromId'] = currentId
    dataT['userToId'] = followers
    
    dataUsers = pd.concat([dataUsers,dataT])
    time.sleep(70) 
    count +=1
    per = round(count*100.0/len(allUsers),1)
    sys.stdout.write("\rTwitter call %s%% complete." % per)
    sys.stdout.flush()



KeyboardInterrupt: 

In [ ]:

fromId = dataUsers['userFromId'].unique()
dataChat = dataUsers[dataUsers['userToId'].apply(lambda x: x in fromId)]



In [ ]:

dataLookup = dataChat[['userFromName','userFromId']]
dataLookup = dataLookup.drop_duplicates()
dataLookup.columns = ['userToName','userToId']
dataCommunity = dataUsers.merge(dataLookup, on='userToId')



In [ ]:

dataCommunity.to_csv('dataCommunity.csv',index = False,encoding='utf-8')



In [ ]:

data=pd.read_csv('dataCommunity.csv')



In [ ]:

import plotly.plotly as py
from plotly.graph_objs import *

import networkx as nx



In [ ]:

data.userFromId=data.userFromId.apply(lambda x: int(x))
data.userToId=data.userToId.apply(lambda x:int(x))



In [ ]:

?zip



In [ ]:

Graph = nx.DiGraph()
Graph.add_nodes_from(data['userFromId'])
Graph.add_edges_from(zip(data['userFromId'],data['userToId']))



In [ ]:

lu=data[['userFromName','userFromId']].drop_duplicates()



In [ ]:

for uid in lu['userFromId']:
    Graph.node[uid]['userName'] = lu['userFromName'][data['userFromId']==uid].values[0]



In [ ]:

nx.draw(Graph, pos=nx.spring_layout(Graph,k=.1600),node_color='blue',edge_color='black')



In [ ]:

from textblob import TextBlob
import nltk



In [ ]:

polarity=0.0
i=0
pn=0
pz=0
pp=0
for t in text:
    blob = TextBlob(t)
    for sentence in blob.sentences:
        i=i+1
        polarity=polarity+sentence.sentiment.polarity
        if sentence.sentiment.polarity<0:
            print("\033[1m"+"negative sentiment"+"\033[0;0m")
            print(sentence)
            print(sentence.sentiment.polarity)
            pn=pn+1
        if sentence.sentiment.polarity>0:
            print("\033[1m"+"positive sentiment"+"\033[0;0m")
            print(sentence)
            print(sentence.sentiment.polarity)
            pp=pp+1
        if sentence.sentiment.polarity==0:
            print("\033[1m"+"neutral sentiment"+"\033[0;0m")
            print(sentence)
            print(sentence.sentiment.polarity)
            pz=pz+1



In [ ]:

globalPol=polarity/i
print("\033[1m Global polority:\033[0;0m ",globalPol)



In [ ]:

print("\033[1m Neutral sentiment:\033[0;0m  ",100*pn/i," %")
print("\033[1m Positive sentiment:\033[0;0m ",100*pp/i,"%")
print("\033[1m Negative sentiment:\033[0;0m ",100*pz/i," %")




<h1 id="Partie-YouTube">Partie YouTube<a class="anchor-link" href="#Partie-YouTube">¶</a></h1>



<h2 id="Importation-des-librairies">Importation des librairies<a class="anchor-link" href="#Importation-des-librairies">¶</a></h2>


In [ ]:

from bs4 import BeautifulSoup as bs
import requests
import pytube
import time
from selenium.webdriver import Chrome
from contextlib import closing
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import sys
from selenium import webdriver
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split 
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline
from subprocess import check_output
from textblob import TextBlob
import nltk
nltk.download('punkt')




<h2 id="Récupération-des-liens">Récupération des liens<a class="anchor-link" href="#Récupération-des-liens">¶</a></h2>



<p>Nous allons récupérer les liens disponibles sur Youtube en relation avec la phrase clé "Donald Trump"</p>


In [ ]:

base = "https://www.youtube.com/results?search_query="
qstring = "donald trump"
r = requests.get(base+qstring)
page = r.text
soup=bs(page,'html.parser')
vids = soup.findAll('a',attrs={'class':'yt-uix-tile-link'})
videolist=[]
for v in vids:
    tmp = 'https://www.youtube.com' + v['href']
    videolist.append(tmp)




<p>Visualisation des liens</p>


In [ ]:

for item in videolist:
    print(item)




<h2 id="Récuperation-des-commentaires">Récuperation des commentaires<a class="anchor-link" href="#Récuperation-des-commentaires">¶</a></h2>


In [ ]:

chrome_options = Options()  
chrome_options.add_argument("--headless")
ListComs=[]
ListDesId=[]
for i in range(len(ListComs)):
    ListDesId.append(i) 
with closing(Chrome(chrome_options=chrome_options)) as driver:
    
    for i in range(len(videolist)):
        
        wait = WebDriverWait(driver,10)
        MyList.append(videolist[i])
        driver.get(videolist[i])
        for item in range(3):
            wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
            time.sleep(3)

        for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#comment #content-text"))):
            ListComs.append(comment.text)
            print(comment.text)
            
       
    




<p><b>Les commentaires sont stockés dans un fichier CSV</b></p>


In [ ]:

data=pd.DataFrame()
data['Id']=ListDesId
data['comments']=ListComs
data.to_csv('Youtube.csv',index = False,encoding='utf-8')




<h2 id="Analyse-Sentimentale">Analyse Sentimentale<a class="anchor-link" href="#Analyse-Sentimentale">¶</a></h2>



<p>Dans cette étape on va voir quels sont les commentaires positifs et quels sont les commentaires négatifs</p>


In [ ]:

polarity=0.0
i=0
neutral=0
positive=0
negative=0
listPos=[]
listNeg=[]
PosPolarity=0.0
NegPolarity=0.0
with open('Youtube.csv', encoding="utf8") as f:
    for s in f:
        blob = TextBlob(s)
        for sentence in blob.sentences:
            i=i+1
            polarity=polarity+sentence.sentiment.polarity
            if sentence.sentiment.polarity<0:
                print("\033[1m"+"negative sentiment"+"\033[0;0m")
                print(sentence)
                print(sentence.sentiment.polarity)
                negative=negative+1
                NegPolarity=NegPolarity+sentence.sentiment.polarity
                listNeg.append(sentence)
            if sentence.sentiment.polarity>0:
                print("\033[1m"+"positive sentiment"+"\033[0;0m")
                print(sentence)
                print(sentence.sentiment.polarity)
                positive=positive+1
                PosPolarity=PosPolarity+sentence.sentiment.polarity
                listPos.append(sentence)
            if sentence.sentiment.polarity==0:
                print("\033[1m"+"neutral sentiment"+"\033[0;0m")
                print(sentence)
                print(sentence.sentiment.polarity)
                neutral=neutral+1




<p><b>Séparer les commentaires positifs et négatifs dans 2 fichiers CSV diffèrents</b></p>


In [ ]:

datapos=pd.DataFrame()
datapos['comments']=listPos
datapos.to_csv('YoutubePositive.csv',index = False,encoding='utf-8')
dataneg=pd.DataFrame()
dataneg['comments']=listNeg
dataneg.to_csv('YoutubeNegative.csv',index = False,encoding='utf-8')



In [ ]:

MydataPos=pd.read_csv('YoutubePositive.csv')
MydataNeg=pd.read_csv('YoutubeNegative.csv')




<h2 id="Traitement-automatique-du-langage-naturel">Traitement automatique du langage naturel<a class="anchor-link" href="#Traitement-automatique-du-langage-naturel">¶</a></h2>



<p>Il s'agit de visualiser les mots qui reviennent dans les commentaires et qui caractérisent une réactions positive ou négative au contenu de la vidéo</p>


In [ ]:

def wordcloud_draw(data, color = 'black'):
    words = ' '.join(data)
    cleaned_word = " ".join([word for word in words.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and not word.startswith('#')
                                and word != 'RT'
                            ])
    wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color=color,
                      width=2500,
                      height=2000
                     ).generate(cleaned_word)
    plt.figure(1,figsize=(13, 13))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
    
print("Positive words")
wordcloud_draw(MydataPos['comments'],'white')
print("Negative words")
wordcloud_draw(MydataNeg['comments'])




<h2 id="Récupération-des-descriptions-des-vidéos">Récupération des descriptions des vidéos<a class="anchor-link" href="#Récupération-des-descriptions-des-vidéos">¶</a></h2>


In [ ]:

desc=[]
for item in videolist:
    s = requests.Session()
    r = s.get(item)
    html = r.text
    soup = bs(html, 'lxml')
    for i in soup.find_all('p', id='eow-description'):
        for br in i.find_all('br'):
            next_sib = br.next_sibling
            desc.append(next_sib)
desc            




<p>Stockage des descriptions dans un fichier CSV</p>


In [ ]:

dataDesc=pd.DataFrame()
dataDesc['text']=desc
dataDesc.to_csv('Descriptions.csv',index = False,encoding='utf-8')




<p>Calcul des polarités(positifs/négatifs) des descriptions des vidéos</p>


In [ ]:

polarity=0.0
i=0
neutral=0
positive2=0
negative2=0
PosPolarity2=0.0
NegPolarity2=0.0
with open('Descriptions.csv', encoding="utf8") as f:
    for s in f:
        blob = TextBlob(s)
        for sentence in blob.sentences:
            i=i+1
            polarity=polarity+sentence.sentiment.polarity
            if sentence.sentiment.polarity<0:
               
                negative2=negative2+1
                NegPolarity2=NegPolarity2+sentence.sentiment.polarity
                
            if sentence.sentiment.polarity>0:
                
                positive2=positive2+1
                PosPolarity2=PosPolarity2+sentence.sentiment.polarity
                
            if sentence.sentiment.polarity==0:
               
                neutral=neutral+1




<h2 id="Comparaison-des-polarités-entre-les-commentaires-et-les-descriptions-des-vidéos">Comparaison des polarités entre les commentaires et les descriptions des vidéos<a class="anchor-link" href="#Comparaison-des-polarités-entre-les-commentaires-et-les-descriptions-des-vidéos">¶</a></h2>



<p>Moyenne de la polarité positive des commentaires</p>


In [ ]:

MoyPosPolarity=PosPolarity/positive
MoyPosPolarity




<p>Moyenne de la polarité négative des commentaires</p>


In [ ]:

MoyNegPolarity=NegPolarity/negative
MoyNegPolarity




<p>Moyenne de la polarité positive des descriptions</p>


In [ ]:

MoyPosPolarity2=PosPolarity2/positive2
MoyPosPolarity2




<p>Moyenne de la polarité négative des descriptions</p>


In [ ]:

MoyNegPolarity2=NegPolarity2/negative2
MoyNegPolarity2




<h4 id="Interprétations">Interprétations<a class="anchor-link" href="#Interprétations">¶</a></h4>



<p>On remarque que la moyenne de la polartité positive dans les descriptions est largement supérieure à celle des commentaires.
On remarque que la moyenne de la polartité négative dans les descriptions est largement supérieure à celle des commentaires.
On peut donc dire que les commentaires et les descriptions ne sont pas proportionnelles et vu que les moyennes des descriptions sont élevés on peut émettre une hypothèse sur le fait que les descriptions des vidéos sont exagérées dans le bon ou le mauvais sens</p>



<h3>Partie Facebook</h3>



<p>Importation des librairies:</p>


In [ ]:

from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import unicodedata
from unidecode import unidecode
import pandas as pd
from textblob import TextBlob
import nltk
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output




<p>Préparation du webdriver:</p>


In [ ]:

chrome_path = r"D:\chromedriver.exe"
driver = webdriver.Chrome(chrome_path)




<p>Connexion au compte facebook:</p>


In [ ]:

driver.get("https://www.facebook.com")
driver.find_element_by_name("email").clear()
driver.find_element_by_name("pass").clear()
driver.find_element_by_name("email").send_keys("insert email here")
driver.find_element_by_name("pass").send_keys("insert pwd here")
driver.find_element_by_id("loginbutton").click()




<p>Récupération des données:</p>


In [ ]:

driver.get("https://www.facebook.com/DonaldTrump/photos/a.10156483516640725/10161764020965725/?type=3&__xts__%5B0%5D=68.ARCA05t9NUHm4QghbZBo55ni01UGzZkS62ao_eTuKzsfiHrYuRBESO5TTmlbdO5puQinluVw6qQEHesP-RDgLffbM-yJvZEMISHIfnzZ3rGGmtmIrjPud_uNMF_tI7DAg8TqXBYHE19h3jOUzgAazOk4KVkX15Lwfm7kosPRjkMRB30Qg9eeBbJT4iGbZGtWXQVknUodgS61lIimbHBlrkt-UwncLmC5u7SPKW1peuHUTGB0gpVIQJYsdaY_RLU5w3uEQeaHz5sNSS-qzPIYG0aBm4p-dhNDhbBxHVmVo-IDCFPEs3lhbauBCBEd6o4dGaoSGqcOv8lexKGQ5sA9&__tn__=-R")



In [ ]:

for i in range(0,10):
    driver.find_elements_by_xpath("//*[contains(text(), 'Voir plus de commentaires')]")[-1].click()
    time.sleep(2)
posts = driver.find_elements_by_class_name("UFICommentBody")




<p>Ecrtiture des données dans un fichier texte:</p>


In [ ]:

for post in posts:
    with open("trump.txt", "a", encoding='utf-8') as document:
        document.write(post.text)



In [ ]:

nltk.download('punkt')




<p>Lecture des données du fichier texte:</p>


In [ ]:

texts=[]
with open('trump.txt', 'r') as f:
    for s in f:
        texts.append(s)
        




<p>Analyse Sentimentale des données avec textblob et nltk:</p>


In [ ]:

polarity=0.0
i=0
neutral=0
positive=0
negative=0
text=[]
sen=[]
for s in texts:
    blob = TextBlob(s)
    for sentence in blob.sentences:
        i=i+1
        polarity=polarity+sentence.sentiment.polarity
        if sentence.sentiment.polarity<0:
            print("\033[1m"+"negative sentiment"+"\033[0;0m")
            print(sentence)
            print(sentence.sentiment.polarity)
            text.append(sentence)
            sen.append('negative')
            negative=negative+1
        if sentence.sentiment.polarity>0:
            print("\033[1m"+"positive sentiment"+"\033[0;0m")
            print(sentence)
            print(sentence.sentiment.polarity)
            text.append(sentence)
            sen.append('positive')
            positive=positive+1
        if sentence.sentiment.polarity==0:
            print("\033[1m"+"neutral sentiment"+"\033[0;0m")
            print(sentence)
            print(sentence.sentiment.polarity)
            neutral=neutral+1
            text.append(sentence)
            sen.append('neutral')
    



In [ ]:

d=[]
d.append(negative)
d.append(positive)
d.append(neutral)
d
df = pd.DataFrame(data=d,index=['Negative','Positive','Neutral'],columns=['count'])
df




<p>Exportation de l'analyse vers un fichier csv:</p>


In [ ]:

dataF=pd.DataFrame(data={'text':text,'sentiment':sen
                         })
dataF
dataF.to_csv('facebookSentiments.csv',index=None)




<p>Lecture du fichier csv:</p>


In [ ]:

fs=pd.read_csv('facebookSentiments.csv')



In [ ]:

fs




<p>Préparation des données:</p>


In [ ]:

data = fs[['text','sentiment']]



In [ ]:

# Splitting the dataset into train and test set
train, test = train_test_split(data,test_size = 0.1)
# Removing neutral sentiments
train = train[train.sentiment != "neutral"]




<p>On a éliminé les sentiments de type "neutral"</p>



<p>Dans cette partie , on va visualiser les mots positifs et les mots négatifs</p>


In [ ]:

train_pos = train[ train['sentiment'] == 'positive']
train_pos = train_pos['text']
train_neg = train[ train['sentiment'] == 'negative']
train_neg = train_neg['text']

def wordcloud_draw(data, color = 'black'):
    words = ' '.join(data)
    cleaned_word = " ".join([word for word in words.split()
                            if 'http' not in word
                                and not word.startswith('@')
                                and not word.startswith('#')
                                and word != 'RT'
                            ])
    wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color=color,
                      width=2500,
                      height=2000
                     ).generate(cleaned_word)
    plt.figure(1,figsize=(13, 13))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
    
print("Positive words")
wordcloud_draw(train_pos,'white')
print("Negative words")
wordcloud_draw(train_neg)



In [ ]:

nltk.download('stopwords')




<p>Stop Word: 
Les mots "stop word" sont des mots qui ne contiennent pas de signification importante quand à leur utilisation dans les requêtes de recherche. Habituellement, ces mots sont filtrés des requêtes de recherche car ils renvoient une grande quantité d'informations inutiles (le, pour, etc...).</p>


In [ ]:

fbcomments = []
stopwords_set = set(stopwords.words("english"))

for index, row in train.iterrows():
    words_filtered = [e.lower() for e in row.text.split() if len(e) >= 3]
    words_cleaned = [word for word in words_filtered
        if 'http' not in word
        and not word.startswith('@')
        and not word.startswith('#')]
    words_without_stopwords = [word for word in words_cleaned if not word in stopwords_set]
    fbcomments.append((words_without_stopwords, row.sentiment))

test_pos = test[ test['sentiment'] == 'positive']
test_pos = test_pos['text']
test_neg = test[ test['sentiment'] == 'negative']
test_neg = test_neg['text']



In [ ]:

# Extracting word features
def get_words_in_fbcomments(fbcomments):
    all = []
    for (words, sentiment) in fbcomments:
        all.extend(words)
    return all

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    features = wordlist.keys()
    return features
w_features = get_word_features(get_words_in_fbcomments(fbcomments))

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in w_features:
        features['contains(%s)' % word] = (word in document_words)
    return features




<p>On a extrait les caractéristiques avec nltk lib, tout d'abord en mesurant une distribution fréquente et en sélectionnant les mots résultants.</p>


In [ ]:

wordcloud_draw(w_features)




<p>On a donc tracé les mots les plus fréquemment distribués. La plupart des mots sont centrés autour de "thank".</p>


In [ ]:

# Training the Naive Bayes classifier
training_set = nltk.classify.apply_features(extract_features,fbcomments)
classifier = nltk.NaiveBayesClassifier.train(training_set)




<p>À l’aide du classifieur Naive Bayle nltk, on a classé les caractéristiques des mots extraits.</p>


In [ ]:

neg_cnt = 0
pos_cnt = 0
for obj in test_neg: 
    res =  classifier.classify(extract_features(obj.split()))
    if(res == 'negative'): 
        neg_cnt = neg_cnt + 1
for obj in test_pos: 
    res =  classifier.classify(extract_features(obj.split()))
    if(res == 'positive'): 
        pos_cnt = pos_cnt + 1
        
print('[Negative]: %s/%s '  % (len(test_neg),neg_cnt))        
print('[Positive]: %s/%s '  % (len(test_pos),pos_cnt))    




<p>=&gt; Score de l'algorithme du classificateur</p>



<h2 id="Interprétation:">Interprétation:<a class="anchor-link" href="#Interprétation:">¶</a></h2>



<p>D'après l'analyse sentimentale , on remarque que la plus part des commentaires reflètent un sentiment neutre de la part des interlocuteurs. Par contre, où on peut faire une interprétation logique c'est en regardant les commentaires renvoyant un sentiment positif ou négatif. En effet, 203 négatifs contre 337 positifs qui peut nous dire que les commentaires virent vers l'approbation des avis et des paroles cités par "Donald Trump".</p>



<p>Dans une deuxième partie, on essayé de visualiser les mots les plus utilisés par les internautes et on a fait une classification qui les séparent entre des mots négatifs et des mots positifs, qui peut certainement aider à caractériser les messages et les commentaires destinés à "Donald Trump".</p>
